In [ ]:
# This notebook tests different bits and pieces that will go into the Fizeau control loop

# created 2018 June 26 by E.S.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import numpy.ma as ma
from scipy import misc, signal
from astropy.io import fits
% matplotlib inline

In [2]:
# set some approximate parameters of the observed grism PSF

sig = 5 # sigma of Gaussian profile in x
length_y = 200 # length in y of the psf

In [3]:
# define a Gaussian for correlating it to a grism-ed PSF in x

def gaussian_x(x, mu, sig):    
    
    shape_gaussian = np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))
        
    return shape_gaussian

In [21]:
# read in a file with a grism PSF

fits_image_filename = "images/lm_180507_007017.fits"
hdul = fits.open(fits_image_filename)
image = hdul[0].data

In [22]:
# generate the Gaussian

mu_x_center = 0.5*image.shape[1] # initially center the probe shape for correlation
x_probe_gaussian = gaussian_x(np.arange(image.shape[1]), mu_x_center, sig)

In [23]:
# generate a top hat for correlating it to a grism-ed PSF in y

y_abcissa = np.arange(image.shape[0])
y_probe_tophat = np.zeros(image.shape[0])
y_probe_tophat[np.logical_and(y_abcissa > 0.5*image.shape[0]-0.5*length_y,
                          y_abcissa <= 0.5*image.shape[0]+0.5*length_y)] = 1

In [24]:
# dampen edge effects

repl_val = np.median(image)
image[0:4,:] = repl_val
image[-5:-1,:] = repl_val
image[:,0:4] = repl_val
image[:,-5:-1] = repl_val

In [25]:
# correlate real PSF and probe shape in x

corr_x = signal.correlate(np.sum(image,axis=0), x_probe_gaussian, mode='same')

In [26]:
# correlate real PSF and probe shape in y

# this essentially integrates along x and subtracts the background shape
profile_y = np.subtract(np.sum(image,axis=1), image.shape[1]*np.median(image,axis=1)) 

corr_y = signal.correlate(profile_y, y_probe_tophat, mode='same')

In [27]:
# find centers of psfs

psf_center_x = np.argmax(corr_x)
psf_center_y = np.argmax(corr_y)